<div style='text-align: center;'>
    <img src='https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSQzJzIHdangJTrH2mFXFgsLjuLCjpfXXwbxg&usqp=CAU' width='100'/>
    <h1>Sharif University of Technology</h1>
    <h2>Natural Language Processing</h2>
    <h3>Final Project</h3>
    <h4>Spoiler classification and summary generation</h4>
    <p><strong>Authors:</strong> Parnian Razavipour, Mobina Salimipanah</p>
</div>
<hr/>


# Dataset Preparation and EDA


In [2]:
pip install datasets

In [3]:
!pip install kaggle

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
!kaggle datasets download -d rmisra/imdb-spoiler-dataset

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/rmisra/imdb-spoiler-dataset
License(s): Attribution 4.0 International (CC BY 4.0)
 99% 328M/331M [00:08<00:00, 62.6MB/s]
100% 331M/331M [00:08<00:00, 42.9MB/s]


In [5]:
!unzip imdb-spoiler-dataset.zip
!ls

Archive:  imdb-spoiler-dataset.zip
  inflating: IMDB_movie_details.json  
  inflating: IMDB_reviews.json       
IMDB_movie_details.json  IMDB_reviews.json  imdb-spoiler-dataset.zip  sample_data


In [6]:
import pandas as pd
movie_details_df = pd.read_json('IMDB_movie_details.json', lines=True)

movie_details_df.head()

,movie_id,plot_summary,duration,genre,rating,release_date,plot_synopsis
0,tt0105112,"Former CIA analyst, Jack Ryan is in England wi...",1h 57min,"[Action, Thriller]",6.9,1992-06-05,"Jack Ryan (Ford) is on a ""working vacation"" in..."
1,tt1204975,"Billy (Michael Douglas), Paddy (Robert De Niro...",1h 45min,[Comedy],6.6,2013-11-01,Four boys around the age of 10 are friends in ...
2,tt0243655,"The setting is Camp Firewood, the year 1981. I...",1h 37min,"[Comedy, Romance]",6.7,2002-04-11,
3,tt0040897,"Fred C. Dobbs and Bob Curtin, both down on the...",2h 6min,"[Adventure, Drama, Western]",8.3,1948-01-24,Fred Dobbs (Humphrey Bogart) and Bob Curtin (T...
4,tt0126886,Tracy Flick is running unopposed for this year...,1h 43min,"[Comedy, Drama, Romance]",7.3,1999-05-07,Jim McAllister (Matthew Broderick) is a much-a...


In [7]:
print(f"Total number of rows in the dataset: {movie_details_df.shape[0]}")

Total number of rows in the dataset: 1572


In [8]:
data = movie_details_df[['plot_summary', 'plot_synopsis']]
data.head()

,plot_summary,plot_synopsis
0,"Former CIA analyst, Jack Ryan is in England wi...","Jack Ryan (Ford) is on a ""working vacation"" in..."
1,"Billy (Michael Douglas), Paddy (Robert De Niro...",Four boys around the age of 10 are friends in ...
2,"The setting is Camp Firewood, the year 1981. I...",
3,"Fred C. Dobbs and Bob Curtin, both down on the...",Fred Dobbs (Humphrey Bogart) and Bob Curtin (T...
4,Tracy Flick is running unopposed for this year...,Jim McAllister (Matthew Broderick) is a much-a...


In [9]:
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = text.replace('\n', ' ')  # Replace newlines with spaces
    text = text.replace('\r', ' ')  # Replace carriage returns with spaces
    text = ''.join([c if c.isalnum() or c.isspace() else ' ' for c in text])  # Remove special characters
    text = ' '.join(text.split())  # Remove extra spaces
    return text

# Apply text cleaning to the dataset
data['plot_summary'] = data['plot_summary'].apply(clean_text)
data['plot_synopsis'] = data['plot_synopsis'].apply(clean_text)

<ipython-input-9-1769cf9f1984>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['plot_summary'] = data['plot_summary'].apply(clean_text)
<ipython-input-9-1769cf9f1984>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['plot_synopsis'] = data['plot_synopsis'].apply(clean_text)


In [10]:
from sklearn.model_selection import train_test_split
train_data, temp_data = train_test_split(data, test_size=0.25, random_state=42)

# Split the remainder into test and validation sets
test_data, val_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# Display the sizes of each dataset
print("Training set size:", len(train_data))
print("Validation set size:", len(val_data))
print("Test set size:", len(test_data))

Training set size: 1179
Validation set size: 197
Test set size: 196


In [11]:
from datasets import Dataset
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments, DataCollatorForSeq2Seq

train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)
test_dataset = Dataset.from_pandas(test_data)

In [12]:
tokenizer = AutoTokenizer.from_pretrained('google/long-t5-local-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/811 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [13]:
import torch  # Import torch for GPU utilization

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [14]:
def tokenize_function(examples):
    model_inputs = tokenizer(
        examples['plot_synopsis'],
        truncation=True,
        padding='max_length',
        max_length=3500,
        return_tensors="pt"
    )
    labels = tokenizer(
        examples['plot_summary'],
        truncation=True,
        padding='max_length',
        max_length=315,
        return_tensors="pt"
    )
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [15]:
print(train_dataset.column_names)


['plot_summary', 'plot_synopsis', '__index_level_0__']


In [16]:
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1179 [00:00<?, ? examples/s]

Map:   0%|          | 0/197 [00:00<?, ? examples/s]

In [17]:
train_dataset = train_dataset.remove_columns(['plot_summary', 'plot_synopsis'])
val_dataset = val_dataset.remove_columns(['plot_summary', 'plot_synopsis'])


In [18]:
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


In [19]:
print("Sample data:", train_dataset[0])

Sample data: {'input_ids': tensor([ 11, 864,  11,  ...,   0,   0,   0]), 'attention_mask': tensor([1, 1, 1,  ..., 0, 0, 0]), 'labels': tensor([   16,   126, 25453,     8,   650,    11,     3,    29,     9,   757,
          131,  9461,    16, 19433,    11,   864,     3,     7,  1836,     7,
           19, 10626,    12,   161,    38,     8,   511,  6165,    13,     8,
         2021,    11,  8732,  1870,   232,     9, 12966,   120,     8,     3,
         6830,   107,   924,    11, 12947,   924,  4297,    13,     8, 22750,
         2934,  3835,    11,   864,  6612,    12,   582,     3,     9,  9994,
           11,  8519,     8,  1004,    38,     3,     9,  7234,   771,  1921,
            8,   166,  6165,     3,    15,    51,  9203,  7786,     7,    11,
          864,    81,     8,  3889,    11, 11177,    13,    70, 23958,  7930,
           11,     8, 22308,     3,    29,    23,  1803,  1691,    11,   864,
           12,  3270,    72, 22538,    21,     8,  1164,    11,   864,  1112,
       

# Model

In [21]:

model = AutoModelForSeq2SeqLM.from_pretrained('google/long-t5-local-base')
model.to(device)  # Move model to GPU

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

Some weights of LongT5ForConditionalGeneration were not initialized from the model checkpoint at google/long-t5-local-base and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

LongT5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): LongT5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): LongT5Block(
        (layer): ModuleList(
          (0): LongT5LayerLocalSelfAttention(
            (LocalSelfAttention): LongT5LocalAttention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): LongT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): LongT5LayerFF(
            (DenseReluDense): LongT5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out

In [22]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [23]:

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    warmup_steps=1000,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-05,
    gradient_checkpointing=True,
    fp16 = False,

)


In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)

In [25]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [26]:
!nvidia-smi


Fri Aug  9 18:41:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0              26W /  70W |   1117MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [27]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1014: FutureWarning: The `device` argument is deprecated and wi

Epoch,Training Loss,Validation Loss
1,14.170500,11.008263
2,5.156700,4.272206


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1014: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1014: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


TrainOutput(global_step=590, training_loss=14.891532425961252, metrics={'train_runtime': 3558.0279, 'train_samples_per_second': 0.663, 'train_steps_per_second': 0.166, 'total_flos': 1.1037700058112e+16, 'train_loss': 14.891532425961252, 'epoch': 2.0})

In [28]:
!pip install rouge_score nltk prettytable


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=9dab34b1f47d1d03fbcf388e4946f7a7111f28f0a554cce8fc215c9de9a6316b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [29]:
import nltk
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
from prettytable import PrettyTable
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer
from datasets import load_metric

# Ensure you have downloaded the nltk datasets used in BLEU score calculation
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [30]:
model_path = "./finetuned_longt5_model"
tokenizer_path = "./finetuned_long_t5_tokenizer"

model.save_pretrained(model_path)
tokenizer.save_pretrained(tokenizer_path)

('./finetuned_long_t5_tokenizer/tokenizer_config.json',
 './finetuned_long_t5_tokenizer/special_tokens_map.json',
 './finetuned_long_t5_tokenizer/spiece.model',
 './finetuned_long_t5_tokenizer/added_tokens.json',
 './finetuned_long_t5_tokenizer/tokenizer.json')

In [32]:
model = AutoModelForSeq2SeqLM.from_pretrained('./finetuned_longt5_model')
tokenizer = AutoTokenizer.from_pretrained('./finetuned_long_t5_tokenizer')
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

LongT5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): LongT5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): LongT5Block(
        (layer): ModuleList(
          (0): LongT5LayerLocalSelfAttention(
            (LocalSelfAttention): LongT5LocalAttention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): LongT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): LongT5LayerFF(
            (DenseReluDense): LongT5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out

In [33]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Metric instances
rouge = load_metric("rouge")
bleu_score = corpus_bleu

<ipython-input-33-1b938ab64e01>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")


The repository for rouge contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/rouge.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [34]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [35]:
def generate_summary(batch):
    inputs = tokenizer(batch["plot_synopsis"], max_length=4000, truncation=True, padding="max_length", return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}  # ensure our data is on the correct device.
    outputs = model.generate(inputs["input_ids"], max_length=315, min_length=20, length_penalty=0.8, num_beams=5, early_stopping=True, no_repeat_ngram_size=3)

    batch["pred_summary"] = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return batch

results = test_dataset.map(generate_summary, batched=True, batch_size=1)



Map:   0%|          | 0/196 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1014: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [36]:
predictions = results["pred_summary"]
references = [ex["plot_summary"] for ex in test_dataset]

In [37]:
rouge_output = rouge.compute(predictions=predictions, references=references, use_stemmer=True)
print("ROUGE Scores:", rouge_output)

ROUGE Scores: {'rouge1': AggregateScore(low=Score(precision=0.1321119235672059, recall=0.1640308327037384, fmeasure=0.1412861543646318), mid=Score(precision=0.1431888238282153, recall=0.17520056515024418, fmeasure=0.15127049699191397), high=Score(precision=0.15474333917333782, recall=0.18642705791867176, fmeasure=0.16089618452780904)), 'rouge2': AggregateScore(low=Score(precision=0.0062126969711021884, recall=0.007685674344560914, fmeasure=0.006636097276845922), mid=Score(precision=0.007412589612371583, recall=0.009181046831736572, fmeasure=0.007816585639373725), high=Score(precision=0.008810348630634146, recall=0.010884846581340049, fmeasure=0.009208982150104468)), 'rougeL': AggregateScore(low=Score(precision=0.09240370160560428, recall=0.1174290073875052, fmeasure=0.09941255314152428), mid=Score(precision=0.10070781435963212, recall=0.12636525758030515, fmeasure=0.10731770903508449), high=Score(precision=0.1087295830882989, recall=0.13464982313360754, fmeasure=0.11434115423801129)), 

In [38]:
pt = PrettyTable()
pt.field_names = ["Metric", "High", "Mid", "Low"]

# Adding rows for each ROUGE metric with high, mid, and low values
for key in ['rouge1', 'rouge2', 'rougeL']:
    high = f"{rouge_output[key].high.fmeasure:.4f}"
    mid = f"{rouge_output[key].mid.fmeasure:.4f}"
    low = f"{rouge_output[key].low.fmeasure:.4f}"
    pt.add_row([key.upper(), high, mid, low])

# Print the table
print(pt)

+--------+--------+--------+--------+
| Metric |  High  |  Mid   |  Low   |
+--------+--------+--------+--------+
| ROUGE1 | 0.1609 | 0.1513 | 0.1413 |
| ROUGE2 | 0.0092 | 0.0078 | 0.0066 |
| ROUGEL | 0.1143 | 0.1073 | 0.0994 |
+--------+--------+--------+--------+


In [ ]:
def generate_summary_from_text(input_text):
    # Tokenize the input text
    inputs = tokenizer(input_text, return_tensors="pt", max_length=4000, truncation=True, padding="max_length").to(model.device)

    # Generate output from the model
    outputs = model.generate(
        inputs['input_ids'],
        max_length=315,
        min_length=20,
        length_penalty=0.8,
        num_beams=5,
        early_stopping=True,
        no_repeat_ngram_size=3
    )

    # Decode the generated summary to readable text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return generated_text



In [ ]:
# Example usage
sample_text = "Jack Ryan (Ford) is on a ""working vacation"" in London with his family. He has retired from the CIA and is a Professor at the US Naval Academy. He is seen delivering a lecture at the Royal Naval Academy in London.Meanwhile, Ryan's wife Cathy and daughter Sally are sightseeing near Buckingham Palace. Sally and Cathy come upon a British Royal Guard, and Sally tries to get the guard to react by doing an improvised tap dance in front of him. She's impressed when the guard, trained to ignore distraction, doesn't react at all, and they leave.As Sally and Cathy walk away from the guard, en route to rendezvous with Ryan, they walk by a stolen cab, in which sit three Ulster Liberation Army terrorists: Kevin O'Donnell, the driver, as well as Sean Miller (Sean Bean) and his younger brother Patrick. The three are loading bullets into their guns as they prepare to carry out a scheduled ambush on Lord William Holmes, British Secretary of State for Northern Ireland and a distant member of the British Royal Family (the cousin of the Queen Mother). As they finish loading their weapons, another terrorist, Annette, radios to them that Lord Holmes is leaving the palace with his wife and son.As Lord Holmes's car leaves the palace, the terrorists' car falls in line behind him. They follow the car, headed for the ambush point. As they drive, O'Donnell and the Miller brothers don ski masks.Meanwhile, Ryan is about to cross the street to meet with his wife and daughter. He arrives at the same time that Lord Holmes's car is passing through the area.Suddenly, another cab, parked on the side of the road, driven by Annette, pulls out and swerves in front of Lord Holmes's vehicle. Simultaneously, O'Donnell's vehicle swerves to a stop behind the car. The Millers and another accomplice jump out and quickly plant a bomb underneath the chauffeured car's engine. Ryan sees what's going on and hastily gets his family to cover just as the bomb is detonated. Moments later, Sean and another accomplice begin shooting into the car, riddling Lord Holmes's driver and bodyguard with bullet. The onslaught of bullets does not kill Lord Holmes, who is currently using his body to shield his family. Sean marches over to the rear door and orders Lord Holmes out of the car at gunpoint. Ryan, unable to stand being just a bystander, immediately breaks cover, runs up behind Miller, and disarms him. As Ryan grabs Sean's pistol, he gets shot in the left shoulder by another terrorist. He shoots one of the other terrorists in the shoulder. Patrick attempts to flee, but Ryan fatally shoots him in the chest. Seeing two of the royal guards racing towards the scene, the remaining terrorists quickly get back in their cars and drive off, leaving Sean to be captured when the authorities arrive. They retreat back to a warehouse, where they change out license plates.While recovering, Ryan is called to testify against Miller as a witness. Subsequently, Miller is convicted on all charges and sentenced to life in prison. Ryan is awarded the order of Knight Commander of the Victorian Order, and eventually returns to the United States.On the possibility that Miller's ULA comrades might try to liberate him, the authorities take no chances. To that extent, they set up decoy convoys on the day that Miller is to be transferred to Albany Prison on the Isle of Wight. However, someone manages to tip off the ULA as to which convoy is carrying Miller. Miller's motorcade is in transit when it comes to a stop at a drawbridge. The raised bridge, however, is a trap meant to block them in. Moments after the convoy comes to a stop, Miller's comrades attack, using rocket launchers to blow up the cars at the front and rear of the convoy. They then march up to the prison van and order the guards out at gunpoint by taking the bridge operator hostage and threatening to shoot him. One of the guards hesitates and is shot fatally. The other officer and the inspector are pulled out, thrown on the ground and forced to lie on their stomachs. O'Donnell hands Miller a pistol, with which he coldly executes both officers and the bridge operator by shooting them in the backs of their heads. They then depart the scene.Miller and his companions flee on a cargo ship from Britain to Libya, to prepare for their next kidnapping attempt on Lord Holmes. Miller however, cannot shake his anger towards Ryan for killing his younger brother and persuades several members of his entourage to accompany him to the United States on a short mission targeting Ryan and his family.Miller travels to the United States, accompanied by Annette, comrade Ned Clark, and a fourth henchman. Annette and Clark travel to the United States Naval Academy to ambush Ryan as he's leaving work. Ryan notices Clark idling nearby. Clark casually walks away, but Ryan clearly unnerved, then gets even more nervous as he hears a car engine starting. His suspicions are well warranted-as he continues walking down the street, he looks in the reflection of a parked van's rearview door and sees that Clark, and the stolen car driven by Annette, are following him. Ryan pretends not to see them until he's walking past a parked panel truck, at which point he ducks behind the truck. Clark draws a silenced pistol and prepares to corner Ryan, but Ryan attacks him first. Clark quickly gets the upper hand and throws Ryan to the curb. He grabs his pistol and is about to shoot Ryan when he's shot dead himself by a US Marine sentry. Annette quickly drives away in her Jeep. Ryan, realizing that Miller is going to target Cathy and Sally, jumps into his car and drives off.Simultaneously, Miller and the other henchman follow Cathy as she picks up Sally from school, with Miller riding shotgun. They prepare to follow the two, but are temporarily held up by a crossing guard. Once the crossing guard clears, they pursue her to US Highway 50. Ryan tries to frantically call Cathy on his car phone, but she is busy answering phone calls from the hospital. As he's trying to connect with her, Miller and his henchman recklessly weave in and out of traffic to catch up with Cathy. Ryan eventually gets through to Cathy and tells her to get off the highway and find the nearest police station. Unfortunately, just as the call ends, Miller's van comes charging up on her using the left shoulder. Miller moves to the backseat of the van, grabs a submachine gun, and slides open the door. Another car momentarily delays him, but once he has a clear aim, Miller opens fire, spraying Cathy's car with bullets. A multicar pileup ensues as Cathy loses control of her car and crashes head-on into a concrete barrier. Ryan, who has found himself stuck in traffic headed in the other direction, sees the smoke from the crash and realizes the worst.Cathy and Sally are airlifted to a nearby hospital for treatment. Enraged over the near-loss of his family, Ryan decides to go back to work for the CIA, having earlier rejected the appeal of his former superior, Vice Admiral James Greer (James Earl Jones).Ryan's tireless work leads him to conclude that Miller has taken refuge in a training camp, one of many located in Libya. He also determines that a recent new arrival at the camp may be one of O'Donnell's English informants, Dennis Cooley. Ryan makes his recommendations to his superiors at the CIA. Greer invites Jack to a live feed of an SAS strike team attack on the camp. Everyone in the camp is eliminated but unbeknownst to the CIA and Ryan, Miller and his companions have already fled the camp and are on their way to the US to stage their next attack on Lord Holmes.Lord Holmes decides to visit Ryan at his home to formally present his KCVO. With the aid of Lord Holmes' traitorous assistant, Miller's group tracks Holmes to this location, kills the Diplomatic Security Service agents and Maryland state troopers guarding the house, and attempts once more to kidnap Lord Holmes. Ryan leads Holmes and his family to safety while he attempts to lure Miller and his companions away from his home.The FBI Hostage Rescue Teams are scrambled to pick up Holmes. Upon realising that Ryan is leading them away from Holmes, Miller's companions try to persuade Miller to turn around, but an enraged and deranged Miller kills his terrorist companions and continues his pursuit of Ryan. Ryan and Miller fight hand to hand; Miller is killed when Ryan impales him backward on a boat anchor, and his body is obliterated in the subsequent explosion of the craft.Credits roll just after Caroline Ryan learns the gender of the child she is going to have, and before she tells Jack and Sally."
generated_summary = generate_summary_from_text(sample_text)
print("Generated Summary:", generated_summary)